# Sequence to Sequence Networks in CNTK

## Background

Traditional neural networks 

<img src=http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png width=500px>


<img src=http://karpathy.github.io/assets/rnn/diags.jpeg width=750px>

## Problem: Grapheme-to-Phoneme Conversion

The [grapheme](https://en.wikipedia.org/wiki/Grapheme) to [phoneme](https://en.wikipedia.org/wiki/Phoneme) problem is a translation task that takes the letters of a word as the input sequence (the graphemes are the smallest units of a writing system) and outputs the corresponding phonemes; that is, the units of sound that make up a language. In other words, the system aims to generate an unambigious representation of how to pronounce a given input word.

### Example

| Letters  | T | A | N | G | L | E |
| --- | --- |
| Phonemes | ~T | ~AE | ~NG | ~G | ~AHL | null |



## Problem: English-to-French Translation

The machine translation problem is well-known and easy to understand: given some English language sentence E (our input sequence), translate it to the equivalent French sentence F (our output sequence). MT researchers have been working on this problem for decades, coming up with ever-more complex systems to eke our the next fraction of a BLEU point. However, sequence-to-sequence networks have in some ways made a lot of their work antiquated (see, e.g. [Neural Machine Translation by Jointly Learning to Align and Translate](http://arxiv.org/abs/1409.0473)).


### Data

The Hansard...


#### Example:
| Language | Sentence | Source |
| --- | --- | --- |
| English | Fueling this growth in royalty revenues is the United States demand, which some day may place our Canadian domestic needs at risk. | hansard.36.2.house.debates.077.e |
| French | Les États-Unis aimeraient bien transformer cette croissance en redevances, ce qui pourrait mettre nos propres besoins en périls. | hansard.36.2.house.debates.077.f |


### Pre-processing and CNTKTextFormat

...

## Step 0: import all of the required functionality...

In [2]:
import numpy as np
import sys
import os
import math
from cntk import Trainer, Axis, text_format_minibatch_source, StreamConfiguration
from cntk.device import cpu, set_default_device
from cntk.learner import momentum_sgd, momentum_schedule
from cntk.ops import input_variable, cross_entropy_with_softmax, classification_error, sequence, slice
from cntk.ops import past_value, future_value, element_select, plus, hardmax
from cntk.ops.functions import CloneMethod

## Step 1: setup the inputs and parameters

In [5]:
# in the g2p problem the vocabs are separate, but for simplicity we share the combined vocab
input_vocab_dim = 69
label_vocab_dim = 69

# network complexity; higher allows network to be more expressive -- but look our for over-fitting
hidden_dim = 128
num_layers = 1

# Source and target inputs to the model
batch_axis = Axis.default_batch_axis()
input_seq_axis = Axis('inputAxis') # create a new named axis for the input
label_seq_axis = Axis('labelAxis') # create a new named axis for the output/label

# Two additional dynamic axes (dynamic because they're only known when the variable is bound to actual data)
# --> the batch axis (the axis along which multiple sequences are batched); and sequence axis (differfent)
input_dynamic_axes = [batch_axis, input_seq_axis]
raw_input = input_variable(shape=(input_vocab_dim), dynamic_axes=input_dynamic_axes, name='raw_input')

label_dynamic_axes = [batch_axis, label_seq_axis]
raw_labels = input_variable(shape=(label_vocab_dim), dynamic_axes=label_dynamic_axes, name='raw_labels')

## Step 2: define the network

As discussed before, the sequence-to-sequence network is, at its most basic, an RNN encoder followed by an RNN decoder, and a dense output layer. We could do this in a few lines with the layers library, but let's go through things in a little more detail without adding too much complexity.

...

## Training

For sequence-to-sequence networks, the loss we use is cross-entropy. 

## Putting it all together



## Attention

An important extension to sequence-to-sequence models, especially when dealing with long sequences, is to use an attention mechanism. The idea behind attention is to allow the decoder, first, to look at any of the hidden state outputs from the encoder (instead of using only the final hidden state), and, second, to learn how much attention to pay to each of those hidden states given the context. This allows the outputted word at each time step `t` to depend not only on the final hidden state and the word that came before it, but instead on a weighted combination of *all* of the input hidden states!

...

The attention vector at output time $t$ over the input words $(1, ..., T_A)$ is defined as:

$$u_i^t = v^T tanh( W_1 h_i + W_2 d_t)$$
$$a_i^t = softmax(u_i^t)$$
$$d'_t = \sum_{i=1}^{T_A}{a_i^t h_i}$$

...

<img src=http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.23.48-PM.png width=550px>

## Greedy decoding

Once we have a trained model, we want 


## Beam search decoding

...